In [1]:
!pip install opencv-python torch torchvision torchaudio numpy ultralytics deep_sort_realtime


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.8/39.5 MB 6.7 MB/s eta 0:00:06
   - -------------------------------------- 1.8/39.5 MB 4.4 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/39.5 MB 3.7 MB/s eta 0:00:10
   --- ------------------------------------ 3.4/39.5 MB 4.2 MB/s eta 0:00:09
   ---- ----------------------------------- 4.2/39.5 MB 4.1 MB/s eta 0:00:09
   ----- ---------------------------------- 5.0/39.5 MB 4.0 MB/s eta 0:00:09
   ----- ---------------------------------- 5.8/39.5 MB 4.1 MB/s eta 0:00:09
   ------ --------------------------------- 6.6/39.5 MB 4.0 MB/s eta 0:00:09
   ------- -------------------------------- 7.3/39.5 MB 4.0 MB/s eta 0:00:09
   -------- ------------------------------- 8.1/39.5 MB 4.0 MB/s eta 0:00:08
   --------- ------------------------------ 8.9/39.5 MB 4.0 MB/s eta 0:00:08
   --------- ------------------------------ 9.7/39.5 MB 3.9 MB/s eta 0:00:08
   ---

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
import os
os.getcwd()

'C:\\Users\\ASUS'

In [6]:
model = YOLO('yolov8n.pt')
tracker = DeepSort(max_age=15)

In [54]:
# Track seen objects
object_history = set()
frame_count = 0

In [56]:
# Store object IDs for detection
current_objects = set()
new_objects_detected = set()
missing_objects_detected = set()


In [92]:
# Open input video
cap = cv2.VideoCapture(r"C:\Users\ASUS\Downloads\input_video.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
ret, frame = cap.read()
if ret:
    print("Video loaded successfully!")
    print("Frame size:", frame.shape)
else:
    print("⚠️ Failed to load video — check the file path.")


Video loaded successfully!
Frame size: (1080, 1920, 3)


In [94]:
out = cv2.VideoWriter("C:/Users/Asus/Downloads/output_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


In [82]:
ret, frame = cap.read()
if ret:
    print("Video loaded successfully!")
    print("Frame size:", frame.shape)
else:
    print("⚠️ Failed to load video — check the file path.")


⚠️ Failed to load video — check the file path.


In [96]:
# Track objects
seen_ids = set()
prev_ids = set()
frame_count = 0


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    results = model(frame)[0]
    detections = []

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        label = model.names[cls]

        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))

    tracks = tracker.update_tracks(detections, frame=frame)

    current_ids = set()

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        bbox = track.to_ltrb()
        label = track.get_det_class()

        current_ids.add(track_id)
        seen_ids.add(track_id)

        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} ID: {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Detect new and missing objects
    new_ids = current_ids - prev_ids
    missing_ids = prev_ids - current_ids

    for nid in new_ids:
        cv2.putText(frame, f'New Object ID: {nid}', (20, 30 + 20 * nid), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    for mid in missing_ids:
        cv2.putText(frame, f'Missing Object ID: {mid}', (20, height - 30 - 20 * mid), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    prev_ids = current_ids.copy()
    out.write(frame)


0: 384x640 2 persons, 4 chairs, 1 dining table, 336.2ms
Speed: 85.4ms preprocess, 336.2ms inference, 52.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 1 dining table, 126.0ms
Speed: 8.7ms preprocess, 126.0ms inference, 10.1ms postprocess per image at shape (1, 3, 384, 640)


In [1]:
cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Done! Output saved as output_video.mp4")

NameError: name 'cap' is not defined